In [29]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [30]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [31]:
peft_model_id = "/home/ysx/src/AI/llm_demo/outputs/checkpoint-1000"
config = PeftConfig.from_pretrained(peft_model_id)


In [32]:

inference_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit=True, 
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.82s/it]


In [33]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
)

In [34]:
model = PeftModel.from_pretrained(inference_model, peft_model_id, device_map="auto")

In [35]:
model = inference_model

In [36]:
model = model.eval()

In [37]:
def generate_prompt(instruction, input):
    return (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    )
   

In [43]:
def make_inference(instruction, input):
  question = generate_prompt(instruction, input)
  batch = tokenizer(question, return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=200)

  print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [39]:
i = "定义五行的生克制化规律。"

In [40]:
t = "五行的生克制化规律是五行结构系统在正常情况下的自动调节机制。五行之间通过相生相克的关系，维持事物的平衡和发展。相生是指五行之间互相滋生和促进的关系，如木生火、火生土等。相克是指五行之间相互制约的关系，如木克土、土克水等。制化关系是生克关系的结合，生中有克、克中有生，相反相成，维持和促进事物相对平衡协调和发展变化。"

In [44]:
make_inference(i, t)

/home/ysx/miniconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1535: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
定义五行的生克制化规律。

### Input:
五行的生克制化规律是五行结构系统在正常情况下的自动调节机制。五行之间通过相生相克的关系，维持事物的平衡和发展。相生是指五行之间互相滋生和促进的关系，如木生火、火生土等。相克是指五行之间相互制约的关系，如木克土、土克水等。制化关系是生克关系的结合，生中有克、克中有生，相反相成，维持和促进事物相对平衡协调和发展变化。

### Response:
五行的生克制化规律是五行理论中描述事物相互作用的基本规律。相生是指五行元素之间的相互滋养和促进，相克是指五行元素之间相互制约和克制的关系。制化关系是生克关系的结合，通过相生和相克的关系，维持和促进五行之间的平衡和发展。仁、义、礼、忠、孝等五种关系在五行理论中对应，体现为五种不同的调节机制，以保证事物的平衡和发展。仁属木，义属火，礼属土，忠属金，孝属水，它们之间存在生克关系，形成五行的调节机制。仁生义，义生礼，礼生忠，忠生孝，孝生仁，反之，义克礼，礼克义，忠克礼，礼克忠，孝克义，义克孝，形成五行之间的相互制约和促进。仁、义、礼


In [42]:
# merge base model and lora model as a standalone model.
# merged_model = model.merge_and_unload()